# Trabalho Prático 2 - Boosting

### Cinthia M. Souza

Neste trabalho é apresentado uma implementação do algoritmo Adaboost. O algoritmo implementado é baseado no artigo intitulado "A Short Introduction to Boosting", dos autores Yoav Freund e Robert E. Schapire.

Nós vamos trabalhar em um problema de classificação considerando que temos:

$(x_{1}, y_{1}), ..., (x_m, y_m)$, onde $x_i \in X, y_{i} \in Y = \{-1, 1\}$

O Algoritmo segue os seguintes passos:

- Inicializa os pesos de $D_{1}(i) = \frac{1}{m}$
- Para t = 1, ..., T  faça:

    - Treina um classificador fraco utilizando a distribuição de pesos $D_{t}$;
    - Dada a hipótese $h_t : X \rightarrow \{-1, 1\}$ com erro:


        $error = \sum_{i:h_{t}(z_{i}) != y_{i}} D_{t}(i)$.

    - Defina o valor de $\alpha_{t}$

    $\alpha_t = \frac{1}{2} ln(\frac{1- error}{error})$.

    - Realiza a atualização dos pesos:

    $ D_{t+1}(i) = \frac{D_{t}(i) exp(-\alpha_{t}y_{i}h_{t}(x_{i}))}{Z_t}$,

onde $Z_{t} é o fator de normalização.

- Cálculo da hipótese final:
    
$H(X) = sign( \sum_{t=1}^{T} \alpha_{t} h_{t}(x))$.

Basicamente, o erro é a soma dos pesos dos exemplos erroneamente classificados. O valor de $\alpha_{t}$ mede a importância que é atribuída a $h_{t}$. A regra de atualização dos pesos tem como objetivo aumentar o peso de exemplos classificados errôneamente e diminuir o peso do exemplos que foram corretamente classificados. Sendo assim, podemos dizer que o peso concentra-se nos exemplos que foram errôneamente classificados. Por fim, a hipótese final H é uma votação entre os classificadores fracos.



# Libraries

In [47]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import label_binarize

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

import plotly.graph_objects as go
from matplotlib import pyplot as plt
import numpy as np

import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

# Implementação AdaBoost

In [48]:
def adaboost(X, y, n_estimators=10):

    stumps = []
    alphas = []
    errors = []
    ada = []

    shape = X.shape[0]
    weights = np.zeros(shape=(n_estimators, shape))

    #Inicialização dos pesos como sendo 1/m
    weights[0] = [1/shape] *shape
    
    for i in range(n_estimators):

        # stump
        stump = DecisionTreeClassifier(max_depth=1)
        stump.fit(X, y, sample_weight=weights[i])
        pred = stump.predict(X)

        #Calculo do erro
        error = sum(weights[i][(pred != y)])

        #Calculando alpha t
        alpha = 1/2 * np.log((1 - error) / error) 

        # Atualização dos pesos
        new_weights = (weights[i] * np.exp(-alpha * y * pred))
        new_weights = new_weights/new_weights.sum()

        if i+1 < n_estimators:
            weights[i+1] = new_weights

        stumps.append(stump)
        errors.append(error)
        alphas.append(alpha)
        ada.append(np.prod(((errors[i]*(1-errors[i]))**1/2)))

    return ada, errors, stumps, alphas

In [49]:
def predict(X, alphas, stumps):
    return np.sign(np.dot(alphas, np.array([stump.predict(X) for stump in stumps])))

# Estimativa do erro esperado usando K-Fold Cross Validation 

Para obter uma estimativa do erro esperado, foi utilizada a técnica cross-validation, com k = 5. 

In [50]:
def cross_validation (X, y, n_estimators=100, nfolds=5):

    acc = []
    roc = []
    f1 = []

    kfold = KFold(nfolds, True, 1)

    X = np.asarray(X)

    for train_index, test_index in kfold.split(X):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        ada, errors, stumps, alphas = adaboost(X_train, y_train, n_estimators=n_estimators)

        y_pred = predict(X_test, alphas, stumps)

        acc.append(accuracy_score(y_test, y_pred))
        roc.append(roc_auc_score(y_test, y_pred))
        f1.append(f1_score(y_test, y_pred))


    return acc, roc, f1

# Carregamento e preparação da base

Foram necessárias realizar duas transformações no conjunto de dados, a primeira foi a de one-hot encoding para binarizar os atributos da base e a segunda foi uma transformação no atributo label. Nessa transformação, os rótulos "positive" receberam o valor "1" e os rótulos "negative" receberam o valor -1.

In [51]:
df = pd.read_csv('tic-tac-toe-endgame.csv')
dummies = pd.get_dummies(df[df.columns[:len(df.columns)-1]],drop_first = True)
dummies

,V1_o,V1_x,V2_o,V2_x,V3_o,V3_x,V4_o,V4_x,V5_o,V5_x,V6_o,V6_x,V7_o,V7_x,V8_o,V8_x,V9_o,V9_x
0,0,1,0,1,0,1,0,1,1,0,1,0,0,1,1,0,1,0
1,0,1,0,1,0,1,0,1,1,0,1,0,1,0,0,1,1,0
2,0,1,0,1,0,1,0,1,1,0,1,0,1,0,1,0,0,1
3,0,1,0,1,0,1,0,1,1,0,1,0,1,0,0,0,0,0
4,0,1,0,1,0,1,0,1,1,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,1,0,0,1,0,1,0,1,1,0,1,0,1,0,0,1,0,1
954,1,0,0,1,1,0,0,1,0,1,1,0,0,1,1,0,0,1
955,1,0,0,1,1,0,0,1,1,0,0,1,0,1,1,0,0,1
956,1,0,0,1,1,0,1,0,0,1,0,1,0,1,1,0,0,1


In [52]:
y = label_binarize(df['V10'], classes=['negative', 'positive'], neg_label=-1, pos_label=1)
y = y.reshape(1,-1)[0]
X = dummies

## Cross-validation

Posteriormente, foi realizada a estimativa do erro esperado utilizando a técnica k-fold crossvalidation com k = 5. Além disso, foram avaliados 6 diferentes valores para o número de estimators. Sendo esses: 10, 20, 30, 100, 200 e 500.

Para avaliar os resultados, foi utilizada as métricas:

- Acurácia: Avalia a quantidade de acertos do algoritmo em relaçao ao total de registros da base;
- F1-score: Média harmônica entre as métricas precision e recall;
- AUC: Representa a área abaixo da curva ROC. O valore de AUC varia entre 0 e 1, onde 0 significa que 100% das predições estão corretas e 1 que 100% das predições estão corretas.

In [53]:
metrics_mean ={'acc': [], 'roc': [], 'f1': []}
metrics_std ={'acc': [], 'roc': [], 'f1': []}

n_estimators = [10, 20, 50, 100, 200, 500]

for i in n_estimators:

    
    acc, roc, f1 = cross_validation (X, y, n_estimators=i, nfolds=5)

    metrics_mean['acc'].append(np.mean(acc))
    metrics_mean['roc'].append(np.mean(roc))
    metrics_mean['f1'].append(np.mean(f1))

    metrics_std['acc'].append(np.std(acc))
    metrics_std['roc'].append(np.std(roc))
    metrics_std['f1'].append(np.std(f1))


A fim de analisar os resultados obtidos, foi representado gráficamente o valor da média e o desvio padrão das métricas nas 5 dobras para cada um dos valores de estimators. Os gráficos a seguir apresentam os resultados obtidos.

In [54]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=n_estimators, y=metrics_mean['acc'],
                    mode='lines+markers',
                    name='accuracy'))
fig.add_trace(go.Scatter(x=n_estimators, y=metrics_mean['roc'],
                    mode='lines+markers',
                    name='auc score'))
fig.add_trace(go.Scatter(x=n_estimators, y=metrics_mean['f1'],
                    mode='lines+markers', name='f1-scores'))
fig.update_layout(title="Variação do valor médio das métricas nas 5 fold em relação ao número de iterações")
fig.show()

In [55]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=n_estimators, y=metrics_std['acc'],
                    mode='lines+markers',
                    name='accuracy'))
fig.add_trace(go.Scatter(x=n_estimators, y=metrics_std['roc'],
                    mode='lines+markers',
                    name='auc score'))
fig.add_trace(go.Scatter(x=n_estimators, y=metrics_std['f1'],
                    mode='lines+markers', name='f1-scores'))
fig.update_layout(title="Variação do valor desvio padrão das métricas nas 5 fold em relação ao número de iterações")
fig.show()

Com base nos dados apresentados nos gráficos acima, é possível concluir que um número bom de estimators é 200. Pois esse possui desempenho superior aos menores e similar aos com quantidade de estimator maior que 200. Sendo assim, nos próximos experimentos, será utilizado o número de estimators igual a 200.

# Variações no desempenho de cada stump após o ajuste de pesos

O objetivo do próximo experimento é avaliar o desempenho de cada um dos 200 stumps criados. Para isso, inicialmente foi realizada a divisão dos dados em treino e teste. Foram separados 20% da base para teste e 80% para treinamento. Posteriormente é realizado o treinamento do adaboost e os valores de alpha e o stump criados em cada iteração são recuperados. 

Após o treinamento é realizada a predição com cada um dos stumps e são calculadas as métricas accuracy, f1-score e AUC.

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)


In [57]:
ada, errors, stumps, alphas = adaboost(X_train, y_train, n_estimators=200)

In [58]:
def predict_2(X, alphas, stumps):
    return  np.array([stump.predict(X) for stump in stumps])

acc = []
roc = []
f1 = []

y_preds = predict_2(X_test, alphas, stumps)

for y_pred in y_preds:

    acc.append(accuracy_score(y_test, y_pred))
    roc.append(roc_auc_score(y_test, y_pred))
    f1.append(f1_score(y_test, y_pred))

In [63]:
fig = go.Figure()

fig.add_trace(go.Violin(y=acc, name='accuracy'))
fig.add_trace(go.Violin(y=f1, name='f1-score'))
fig.add_trace(go.Violin(y=roc, name='auc'))

fig.update_layout(title="Desempenho do classificador a cada ajuste de peso e treinamento")

fig.show()

A partir do gráfico acima é possível ver que a métrica accuracy e auc estão concentradas próximo ao valor de 0.5. Já a métrica f1 possui está concentrada em 0.68, porém, diferentemente da accuracy e auc ela possui módelos com métrica 0. O que chama bastante atenção. 

Vamos comparar agora o desempenho médio de cada modelo com o desempenho do modelo final, que utiliza a votação entre os stumps.

## Avaliação do desempenho na base de test

In [60]:
y_pred = predict(X_test, alphas, stumps)

print("Accuracy: {} ".format(accuracy_score(y_test, y_pred)))
print("AUC: {} ".format(roc_auc_score(y_test, y_pred)))
print("F1: {} ".format(f1_score(y_test, y_pred)))

Accuracy: 0.9635416666666666 
AUC: 0.9512238805970149 
F1: 0.9725490196078431 


No gráfico apresentada acima foi possível ver que, em geral, os stumps possuem um desempenho de 0.5 para Acurácia e AUC e de aproximadamente 0.68 para o F1-score. Oque é um desempenho muito ruim. Contudo, quando agregamos os stumps e realizamos a predição como sendo a votação o desempenho obtido é quase duas vezes melhor que o de 1 stump. O stump de melhor desempenho apresentou uma acurácia de 0.68, f1-score de 0.75 e AUC de 0.66. Quando combinamos os stumps constata-se que a combinação dos stumps gera um resultado superior que o melhor stump obtido.

Além disso, com base nos resultados obtido na base de teste, verifica-se que a estimativa de erro obtida com cross-validation foi muito precisa, com diferença menor que 1%.

# Referências

Freund, Y., Schapire, R., & Abe, N. (1999). A short introduction to boosting. Journal-Japanese Society For Artificial Intelligence, 14(771-780), 1612.